In [3]:
import sys,os
import cv2
import caffe
import numpy as np

In [4]:
net_file = '/home/yaowanchao/code/MobileNet-SSD/MobileNetSSD_deploy_small.prototxt'
caffe_model = '/home/yaowanchao/code/MobileNet-SSD/ZY_MobileNetSSD_deploy_small.caffemodel'
test_dir = '/home/zhongxuexian/20180814.avi'

if not os.path.exists (caffe_model):
    print "caffemodel does not exist"
    exit()

net = caffe.Net(net_file, caffe_model, caffe.TEST)


In [18]:
import copy
CLASSES = ('background', 'car', 'bus', 
           'track', 'person', 'bicycle', 'motor','tricycle','block')
cap = cv2.VideoCapture(test_dir)
video_dir='/home/zhongxuexian/20180814_match.avi'
fps=cap.get(cv2.CAP_PROP_FPS)
size=(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
videoWriter=cv2.VideoWriter(video_dir, cv2.VideoWriter_fourcc('M','J','P','G'),fps,size)
flag=False
cnt=0
used_tracked_id=0

while(cap.isOpened()):
    print "=========================="
    print cnt
    if flag==True:
        pre_bbox_feature=bbox_feature.copy()
        pre_track_id=track_id.copy()
    ret,frame = cap.read()
    if ret==False:
        break
    img = frame
    mean = (104, 117, 123)
    img1 = img.copy()
    img = (img - mean)/127.5
    img = img.astype(np.float32)
    img = img.transpose((2,0,1))
    
    net.blobs['data'].data[...] = img[:,80:480,:]
    out = net.forward()
    feat_name = 'conv0'
    feat_data = net.blobs[feat_name].data
    detect_out_name = 'detection_out'
    bbox_data = net.blobs[detect_out_name].data
    bbox = np.empty([bbox_data.shape[2],4],dtype=int)
    k=0
    for i in range(0,bbox_data.shape[2]):
        if bbox_data[0,0,i,2]>0.29 and bbox_data[0,0,i,5]-bbox_data[0,0,i,3]>45.0/640.0 and bbox_data[0,0,i,6]-bbox_data[0,0,i,4]>30.0/400.0 and bbox_data[0,0,i,3]>40.0/640.0 and bbox_data[0,0,i,4]>20.0/400.0 and bbox_data[0,0,i,5]<600.0/640.0 and bbox_data[0,0,i,6]<360.0/400.0:
            bbox[k,0]=(bbox_data[0,0,i,3]*img1.shape[1]).astype(np.int)
            bbox[k,1]=(bbox_data[0,0,i,4]*(img1.shape[0]-80)).astype(np.int)
            bbox[k,2]=(bbox_data[0,0,i,5]*img1.shape[1]).astype(np.int)
            bbox[k,3]=(bbox_data[0,0,i,6]*(img1.shape[0]-80)).astype(np.int)
            print bbox[k,:]
            k=k+1
        #print bbox_data[0,0,i,3]
        #print bbox_data[0,0,i,4]
        #print bbox_data[0,0,i,5]
        #print bbox_data[0,0,i,6]
        #print "++++"
        #print bbox[k-1,2]
        #print bbox[k-1,0]
        #print bbox[k-1,3]
        #print bbox[k-1,1]
        #print k
    bbox=bbox[0:k,0:4]
    bbox_feature = np.zeros([feat_data.shape[1],bbox.shape[0],2400])
    for i in range(0,bbox.shape[0]):
        for k in range(0,feat_data.shape[1]):
            fea_tmp=feat_data[0,k,int(bbox[i,1]/2):int(bbox[i,3]/2),int(bbox[i,0]/2):int(bbox[i,2]/2)]
            #new_tmp=np.resize(fea_tmp, (400))
            cv_tmp=cv2.resize(fea_tmp,(40,60))
            #print "after_resize"
            #print new_tmp
            #bbox_feature[k,i,:]=new_tmp
            bbox_feature[k,i,:]=np.resize(cv_tmp,(2400))
    if flag == False:
        track_id=np.arange(0,bbox_feature.shape[1])
        used_tracked_id=bbox_feature.shape[1]
        print track_id
        flag=True
    else:
        dist = np.zeros([bbox_feature.shape[1], pre_bbox_feature.shape[1]])
        for i in range(0,bbox_feature.shape[1]):
            for j in range(0,pre_bbox_feature.shape[1]):
                #bbox_noc=np.resize(bbox_feature[:,i,:],(bbox_feature.shape[0]*bbox_feature.shape[2]))
                #pre_bbox_noc=np.resize(pre_bbox_feature[:,j,:],(pre_bbox_feature.shape[0]*pre_bbox_feature.shape[2]))
                #va=np.mat(bbox_noc)
                #vb=np.mat(pre_bbox_noc)
                #numerator=float(va*vb.T)
                #denum=np.linalg.norm(va)*np.linalg.norm(vb)
                #if denum==0:
                    #dist[i,j]=0
                #else:
                    #cos=numerator/denum
                    #dist[i,j]=0.5+0.5*cos
                #print str(i)+' '+str(j)+' '+str(dist[i,j])
                for k in range(0,bbox_feature.shape[0]):
                    #va=np.mat(bbox_feature[k,i,:])
                    #meana=np.mean(va)
                    #vb=np.mat(pre_bbox_feature[k,j,:])
                    #meanb=np.mean(vb)
                    #d1=np.greater_equal(va,meana)
                    #d2=np.greater_equal(vb,meanb)
                    #numerator=float(va*vb.T)
                    #denum=np.linalg.norm(va)*np.linalg.norm(vb)
                    #if denum ==0:
                        #continue
                    #cos=numerator/denum
                    #dist[i,j]+=(0.5+0.5*cos)
                    #dist[i,j]+=np.linalg.norm(bbox_feature[k,i,:]-pre_bbox_feature[k,j,:],ord=1)
                    dist[i,j]+=np.linalg.norm(bbox_feature[k,i,:]-pre_bbox_feature[k,j,:])
                print str(i)+' '+str(j)+' '+str(dist[i,j])
        idx=np.zeros([bbox_feature.shape[1], pre_bbox_feature.shape[1]])
        for i in range(0,dist.shape[0]):
            idx[i,:]=np.argsort(dist[i,:])
        track_id=np.ones([bbox_feature.shape[1]])
        track_id=-track_id
        matched=np.arange(0,bbox_feature.shape[1])
        match_idx=np.ones([2,pre_bbox_feature.shape[1]])
        match_idx=-match_idx
        while not len(matched)==0:
            next_matched=[]
            for i in matched:
                if idx.shape[1]==0:
                    continue
                if idx[int(i),idx.shape[1]-1]==-1:
                    continue
                for k in range(0,idx.shape[1]):
                    if not idx[int(i),k]==-1:
                        max_idx=idx[int(i),k]
                        idx[int(i),k]=-1
                        break
                if match_idx[0,int(max_idx)]==-1:
                    track_id[int(i)]=pre_track_id[int(max_idx)]
                    match_idx[1,int(max_idx)]=dist[int(i),int(max_idx)]
                    match_idx[0,int(max_idx)]=i
                else:
                    if dist[int(i),int(max_idx)]<dist[int(match_idx[0,int(max_idx)]),int(max_idx)]:
                        matched=np.insert(matched, 0,match_idx[0,int(max_idx)])
                        track_id[int(i)]=pre_track_id[int(max_idx)]
                        match_idx[1,int(max_idx)]=dist[int(i),int(max_idx)]
                        match_idx[0,int(max_idx)]=i
                        track_id[int(match_idx[0,int(max_idx)])]=-1
                    else:
                        next_matched=np.append(next_matched,i)
            matched=copy.deepcopy(next_matched)
        for i in range(0,track_id.shape[0]):
            if track_id[i]==-1:
                track_id[i]=used_tracked_id
                used_tracked_id=used_tracked_id+1;
        print track_id
        font =cv2.FONT_HERSHEY_SIMPLEX
        for i in range(0, track_id.shape[0]):
            img1=cv2.rectangle(img1, (bbox[i,0],bbox[i,1]+80),(bbox[i,2],bbox[i,3]+80),(0,255,0),1)
            cv2.putText(img1,'id'+str(track_id[i]),(bbox[i,0],bbox[i,1]+80),font,1,(0,0,255))
            #frame=cv2.rectangle(frame,(int(bbox[i,0]/640.0*frame.shape[0]),int(bbox[i,1]/400.0*frame.shape[1])),(int(bbox[i,2]/640.0*frame.shape[0]),int(bbox[i,3]/400.0*frame.shape[1])),(0,255,0),1)
            #cv2.putText(frame,'id'+str(track_id[i]),(int(bbox[i,0]/640.0*frame.shape[0]),int(bbox[i,1]/400.0*frame.shape[1])),font,1,(0,0,255))
        #cv2.imwrite(str(cnt)+".jpg",img1)
    videoWriter.write(img1)
    cnt=cnt+1
videoWriter.release()
print 'finish'
                

0
[]
1
[]
2
[]
3
[]
4
[]
5
[]
6
[]
7
[]
8
[]
9
[235 141 455 245]
[0.]
10
[237 138 452 245]
0 0 558.237038656429
[0.]
11
[240 129 469 258]
0 0 866.7879073560923
[0.]
12
[]
13
[]
14
[229 140 464 242]
[1.]
15
[235 131 458 243]
0 0 767.4615381662069
[1.]
16
[229 138 466 243]
0 0 787.3326271182334
[1.]
17
[230 139 462 242]
0 0 482.2766518375002
[1.]
18
[251 128 444 242]
[ 43 133 280 247]
0 0 991.7705143083857
1 0 1130.151176577926
[1. 2.]
19
[233 140 463 241]
0 0 1024.8166490915341
0 1 1145.9925592301392
[1.]
20
[217 138 452 241]
0 0 602.6162042396528
[1.]
21
[164 140 404 249]
0 0 720.6620183613301
[1.]
22
[157 136 392 250]
[ 44 158 277 249]
0 0 796.8945282805671
1 0 875.2743061158151
[1. 3.]
23
[]
24
[]
25
[]
26
[]
27
[]
28
[]
29
[366 140 501 196]
[4.]
30
[]
31
[]
32
[]
33
[]
34
[ 51 161 194 250]
[5.]
35
[]
36
[]
37
[ 75 155 250 298]
[ 43 163 110 237]
[6. 7.]
38
[148 142 332 301]
[ 89 163 184 245]
0 0 1088.4403417558583
0 1 1306.5319930712656
1 0 1264.2569914662424
1 1 951.0596455491018
[6

[]
73
[]
74
[]
75
[106 177 177 243]
[170 166 252 225]
[25. 26.]
76
[161 169 262 249]
0 0 968.5455274086022
0 1 1206.8545718597986
[25.]
77
[ 54 163 186 246]
[148 170 245 225]
[245 171 309 233]
0 0 1167.1384756270486
1 0 1021.9878158571897
2 0 1066.6261717198392
[25. 27. 28.]
78
[ 94 165 199 241]
[144 167 247 221]
[482 167 566 231]
0 0 889.9415697073265
0 1 908.9986344255191
0 2 1108.0487795803842
1 0 1018.1942670564067
1 1 1107.507726899351
1 2 1296.114536530363
2 0 982.9593417840379
2 1 936.1488140202173
2 2 1040.298101421031
[25. 28. 27.]
79
[435 139 539 220]
[ 91 162 194 223]
[283 171 330 238]
0 0 919.0668753391882
0 1 1088.4861854837427
0 2 906.5785497140706
1 0 1029.040018889289
1 1 1060.4865537502387
1 2 1025.3026646388535
2 0 1022.0131797140244
2 1 1314.2955710791318
2 2 1109.21458665285
[27. 28. 25.]
80
[ 76 160 201 234]
[451 150 554 231]
0 0 1066.870376374529
0 1 1061.1220094374307
0 2 1301.1962299538682
1 0 988.365469320303
1 1 1064.588119412116
1 2 1283.4374538638697
[28. 27

[272 159 377 235]
[118 169 195 218]
[460 165 524 212]
0 0 556.7876082918115
0 1 765.6600128936917
0 2 828.7271444672116
1 0 783.9827371814469
1 1 832.2868859247224
1 2 784.8922376081873
2 0 796.824322092016
2 1 736.2292518654598
2 2 919.9961306602531
[46. 49. 48.]
166
[275 162 372 222]
[ 42 166 119 215]
[460 159 532 201]
[428 154 506 195]
0 0 774.8703453916719
0 1 877.1728298377942
0 2 899.6475160509808
1 0 656.822438578127
1 1 668.5953558685374
1 2 706.6008610668113
2 0 903.4080466844877
2 1 967.0396789314241
2 2 770.2089036472552
3 0 890.763933834023
3 1 941.9670239892985
3 2 925.7087394953751
[46. 50. 48. 49.]
167
[273 160 366 218]
[475 156 530 199]
0 0 637.8992219011104
0 1 710.9299460457428
0 2 914.7620372286422
0 3 938.2669449679283
1 0 1003.4816683197351
1 1 848.653956907078
1 2 797.2879341037841
1 3 1040.3170804807462
[46. 48.]
168
[262 157 353 218]
0 0 737.5267595224649
0 1 919.6191196051576
[46.]
169
[441 128 594 249]
[250 157 336 219]
[329 155 387 186]
0 0 749.5363208056705


[221 164 320 233]
0 0 460.9972899862697
0 1 480.88035054983203
[67.]
220
[204 157 324 232]
[ 44 167 114 216]
0 0 433.79040339077574
1 0 417.94239517605774
[67. 71.]
221
[228 157 329 227]
[195 159 292 219]
0 0 496.28054935556537
0 1 445.63669270320815
1 0 527.0408133563465
1 1 494.2995977739757
[71. 67.]
222
[265 150 343 216]
[181 154 251 203]
0 0 496.48904446078683
0 1 525.8968962907156
1 0 657.6959083210579
1 1 605.6850998675229
[71. 67.]
223
[288 168 370 229]
[187 168 275 218]
0 0 493.0555718745264
0 1 670.7455883130548
1 0 728.7805169842597
1 1 733.246618396554
[71. 67.]
224
[304 171 388 227]
[171 169 258 226]
[ 60 174 148 218]
[276 175 354 217]
[241 177 287 214]
0 0 483.41133120086005
0 1 727.9497466344817
1 0 518.6814922315147
1 1 689.9568872626558
2 0 528.6875011341434
2 1 733.5217821845763
3 0 698.8169102558777
3 1 795.2585131804337
4 0 776.1470276439042
4 1 834.5029120000713
[71. 67. 72. 73. 74.]
225
[320 148 410 216]
[141 147 250 213]
[179 154 276 208]
[392 155 473 199]
0 0 44

[273 120 392 235]
0 0 262.22820959446165
[92.]
278
[ 63 122 263 291]
[280 114 404 239]
0 0 575.176920250848
1 0 246.4630337213765
[92. 94.]
279
[ 62 132 264 297]
[274 117 402 244]
0 0 524.5480490003071
0 1 612.5344362101648
1 0 556.6265354562794
1 1 325.57429999266475
[92. 94.]
280
[272 118 410 255]
[ 69 134 269 294]
[435 144 597 257]
0 0 598.4047819510055
0 1 308.0064380172858
1 0 462.3224786076488
1 1 580.4869536722526
2 0 650.9874141885728
2 1 469.371517618853
[94. 92. 95.]
281
[295 122 425 256]
[441 144 576 254]
[ 81 141 273 296]
0 0 328.8934353676688
0 1 586.34502104706
0 2 491.11624569335595
1 0 608.6162163050036
1 1 741.5960642997602
1 2 595.6414068343248
2 0 485.6357872627833
2 1 551.6507258997237
2 2 549.0558282018222
[94. 95. 96.]
282
[297 128 436 255]
[ 63 118 301 305]
0 0 342.7789219159464
0 1 600.5389438692865
0 2 504.348310173976
1 0 536.0703743630014
1 1 693.9334937352563
1 2 602.4868275844606
[94. 96.]
283
[285 121 425 250]
[ 50 129 276 285]
[439 142 599 253]
0 0 296.18

...

[376 163 567 261]
0 0 1070.589632551689
[446.]
1644
[316 179 583 308]
[216 204 276 236]
0 0 1161.8989711127047
1 0 1160.4393296710507
[446. 448.]
1645
[381 149 573 251]
0 0 761.302575661766
0 1 1073.782777785462
[446.]
1646
[319 168 593 306]
[208 182 480 304]
0 0 1000.3028209334901
1 0 1189.832159664435
[446. 449.]
1647
[298 151 517 249]
0 0 1067.3945081337
0 1 1258.9323839332542
[446.]
1648
[]
1649
[ 84 191 338 307]
[432 171 547 249]
[450. 451.]
1650
[166 143 399 261]
0 0 737.354153460417
0 1 686.4397503185148
[451.]
1651
[478 187 568 258]
0 0 700.5905304974838
[451.]
1652
[ 97 204 201 264]
[193 161 398 334]
[ 43 206 131 260]
[387 196 467 257]
0 0 654.9525148264394
1 0 591.6000530844117
2 0 631.8773360639838
3 0 651.016660465153
[451. 452. 453. 454.]
1653
[159 212 257 263]
[242 176 444 312]
0 0 570.7706493678896
0 1 661.5834457373342
0 2 615.2335246207332
0 3 740.2821631030814
1 0 645.9819991109847
1 1 516.007385637834
1 2 616.0004224322228
1 3 646.4016809364672
[451. 452.]
1654
[127 

[311 198 371 247]
0 0 459.46180842383376
0 1 540.9989714089587
0 2 707.8929873290058
[477.]
1727
[]
1728
[339 201 385 232]
[480.]
1729
[ 93 173 228 286]
0 0 929.5928683249563
[480.]
1730
[130 185 232 271]
0 0 850.6793912325259
[480.]
1731
[160 186 259 269]
0 0 891.4238567802446
[480.]
1732
[ 71 181 236 296]
[236 191 336 257]
0 0 980.4080541545995
1 0 951.5181378463286
[480. 481.]
1733
[324 161 400 215]
[158 166 311 251]
0 0 1065.2955422614957
0 1 990.5847724259412
1 0 1129.481400397768
1 1 1105.0833458531304
[481. 480.]
1734
[313 177 436 251]
[447 173 515 227]
[ 98 191 175 233]
[247 168 337 222]
[296 174 400 229]
0 0 1138.8440644471934
0 1 1013.7418232967261
1 0 1095.4170944243622
1 1 1177.0426505831442
2 0 1236.713405947802
2 1 1226.947007334428
3 0 1200.7246159237786
3 1 1212.6478465545135
4 0 1439.3205965446623
4 1 1448.9239108377953
[480. 481. 482. 483. 484.]
1735
[473 197 578 256]
[ 57 197 172 255]
[ 44 217 101 248]
0 0 1048.6215327433172
0 1 993.558854890841
0 2 1103.271240015057

[226 171 428 310]
[145 167 249 253]
0 0 468.9977096720979
0 1 668.8712426662048
1 0 607.0173173237818
1 1 601.0007970509233
[504. 505.]
1828
[184 163 453 353]
0 0 612.9774370241955
0 1 501.6465229917003
[505.]
1829
[ 59 180 192 291]
0 0 578.2392478643924
[505.]
1830
[]
1831
[]
1832
[]
1833
[]
1834
[]
1835
[]
1836
[]
1837
[]
1838
[]
1839
[]
1840
[]
1841
[]
1842
[]
1843
[]
1844
[]
1845
[]
1846
[]
1847
[]
1848
[]
1849
[]
1850
[]
1851
[]
1852
[]
1853
[]
1854
[]
1855
[]
1856
[]
1857
[]
1858
[]
1859
[]
1860
[]
1861
[]
1862
[]
1863
[]
1864
[]
1865
[]
1866
[]
1867
[]
1868
[]
1869
[ 98 165 312 343]
[507.]
1870
[]
1871
[]
1872
[]
1873
[]
1874
[]
1875
[]
1876
[]
1877
[]
1878
[]
1879
[]
1880
[]
1881
[]
1882
[]
1883
[]
1884
[]
1885
[]
1886
[]
1887
[]
1888
[]
1889
[]
1890
[]
1891
[]
1892
[]
1893
[]
1894
[]
1895
[]
1896
[]
1897
[]
1898
[]
1899
[]
1900
[]
1901
[]
1902
[]
1903
[]
1904
[]
1905
[]
1906
[]
1907
[]
1908
[]
1909
[]
1910
[]
1911
[]
1912
[]
1913
[]
1914
[]
1915
[]
1916
[]
1917
[]
1918
[]
1919

[]
1989
[]
1990
[]
1991
[]
1992
[]
1993
[]
1994
[]
1995
[]
1996
[]
1997
[]
1998
[]
1999
[]
2000
[]
2001
[]
2002
[]
2003
[]
2004
[]
2005
[]
2006
[]
2007
[]
2008
[]
2009
[]
2010
[]
2011
[]
2012
[]
2013
[]
2014
[]
2015
[]
2016
[]
2017
[]
2018
[]
2019
[]
2020
[]
2021
[]
2022
[]
2023
[]
2024
[]
2025
[]
2026
[]
2027
[]
2028
[]
2029
[]
2030
[]
2031
[]
2032
[]
2033
[]
2034
[]
2035
[]
2036
[]
2037
[]
2038
[]
2039
[]
2040
[]
2041
[]
2042
[]
2043
[]
2044
[]
2045
[]
2046
[]
2047
[]
2048
[]
2049
[]
2050
[]
2051
[]
2052
[]
2053
[]
2054
[]
2055
[]
2056
[]
2057
[]
2058
[]
2059
[]
2060
[]
2061
[]
2062
[]
2063
[]
2064
[]
2065
[]
2066
[]
2067
[]
2068
[]
2069
[]
2070
[]
2071
[]
2072
[]
2073
[]
2074
[]
2075
[]
2076
[]
2077
[]
2078
[]
2079
[]
2080
[]
2081
[]
2082
[]
2083
[]
2084
[]
2085
[]
2086
[143 155 457 356]
[520.]
2087
[165 160 464 359]
0 0 433.2975742172841
[520.]
2088
[193 166 462 344]
0 0 511.18048121497077
[520.]
2089
[233 174 458 341]
[101 194 227 284]
0 0 544.7637890531909
1 0 722.4963739512107
[

[]
2123
[411 185 458 220]
[534.]
2124
[439 185 498 222]
0 0 611.7433623457459
[534.]
2125
[473 200 532 237]
0 0 518.46025293187
[534.]
2126
[493 195 549 230]
0 0 558.0504018947274
[534.]
2127
[494 200 539 230]
0 0 427.937894954682
[534.]
2128
[]
2129
[]
2130
[408 204 463 235]
[535.]
2131
[]
2132
[ 80 180 169 230]
[536.]
2133
[233 195 280 226]
0 0 523.5656249962877
[536.]
2134
[]
2135
[ 92 200 141 232]
[537.]
2136
[]
2137
[]
2138
[]
2139
[]
2140
[]
2141
[]
2142
[]
2143
[]
2144
[]
2145
[]
2146
[]
2147
[]
2148
[]
2149
[]
2150
[]
2151
[]
2152
[]
2153
[]
2154
[]
2155
[]
2156
[]
2157
[]
2158
[]
2159
[]
2160
[]
2161
[]
2162
[]
2163
[]
2164
[]
2165
[]
2166
[]
2167
[]
2168
[]
2169
[]
2170
[]
2171
[]
2172
[]
2173
[]
2174
[]
2175
[]
2176
[]
2177
[]
2178
[]
2179
[]
2180
[]
2181
[]
2182
[]
2183
[]
2184
[]
2185
[]
2186
[]
2187
[]
2188
[]
2189
[]
2190
[295 197 371 249]
[538.]
2191
[289 195 383 269]
0 0 715.686058143898
[538.]
2192
[277 189 394 293]
0 0 610.9041714351102
[538.]
2193
[240 180 417 316]
